In [43]:
from xgboost import XGBRegressor
import sklearn
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
import sklearn.ensemble, sklearn.ensemble, sklearn.svm

In [114]:
data = pd.read_csv("ready.csv")

data = data.replace(to_replace="N/", value=np.NAN)

X = data.drop(columns="rating")
y = data["rating"]
X = X[~y.isna()]
y = y[~y.isna()].to_numpy()

X_copy = X.copy()
y_copy = y.copy()

X = X.iloc[:,3:]
knnI = KNNImputer(missing_values=np.NAN)
X = knnI.fit_transform(X)

s = sklearn.preprocessing.StandardScaler()
X = s.fit_transform(X)

X_tr, X_te, y_tr, y_te = sklearn.model_selection.train_test_split(X, y, train_size=0.75)

In [115]:
xgb = XGBRegressor()
xgb.fit(X_tr, y_tr)
xgb.score(X_te, y_te)

0.6126384910033894

In [116]:
rF = sklearn.ensemble.RandomForestRegressor()
rF.fit(X_tr, y_tr)
rF.score(X_te, y_te)

0.6412685591583309

In [96]:
lR = sklearn.linear_model.LinearRegression()
lR.fit(X_tr, y_tr)
lR.score(X_te, y_te)

0.5483423260440254

In [97]:
svr = sklearn.svm.SVR()
svr.fit(X_tr, y_tr)
svr.score(X_te, y_te)

0.5559755026564499

In [98]:
xgb = XGBRegressor()
xgb.fit(X_tr, y_tr)
scores = xgb.predict(X)

In [82]:
final = pd.concat((X_copy, pd.DataFrame(y_copy, columns=["rating"]), pd.DataFrame(scores, columns=["score"])), axis=1)
final = final.sort_values("score", ascending=False).drop_duplicates("name")

final["total_tags"] = [sum(final[final.columns[6:-2]].iloc[i]) for i in range(final.shape[0])]
final[final.total_tags > 5]

,name,department,school,take_again,difficulty,avg_gpa,Clear grading criteria,Respected,Get ready to read,Skip class? You won't pass.,...,Inspirational,Amazing lectures,Caring,Gives good feedback,Participation matters,Hilarious,Lots of homework,rating,score,total_tags
1921,"YOUSSEFPOUR, H.",Mathematics,UC Irvine,92,3.9,2.609063,5,6,0,0,...,0,0,3,0,0,3,0,5.3,5.301156,29
1086,"YOUSEFI, M.",Political Science,UC Irvine,100,2.0,3.603333,3,0,0,0,...,3,0,0,4,4,0,0,5.0,5.182596,14
810,"MANIAR, R.",Foreign Languages,UC Irvine,100,2.8,3.661935,0,3,0,0,...,0,0,5,0,3,0,0,4.6,5.135962,11
907,"SMITH, J.",Languages,UC Irvine,100,2.3,3.605128,4,0,6,0,...,3,3,0,4,0,0,0,4.1,5.128323,20
2051,"BROADBENT, P.",Humanities,UC Irvine,100,2.9,3.481818,4,4,0,0,...,6,0,0,13,8,10,0,5.1,5.127894,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,"HIRSHLEIFER, S.",Accounting,UC Irvine,0,4.8,3.307143,0,0,7,0,...,0,0,0,0,3,0,0,1.3,1.333407,10
1244,"PREECE, D.",Biomedical,UC Irvine,0,4.5,3.083333,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1.188337,6
1129,"QUIROZ, B.",Education,UC Irvine,0,4.6,3.185000,0,0,6,0,...,0,0,0,0,0,0,0,1.2,1.166928,19
1547,"RADOVIC, A.",Biology,UC Irvine,0,4.5,3.060000,0,0,6,4,...,0,0,0,0,0,0,0,1.0,1.045182,36


In [117]:
final.to_csv("final_regression.csv", index=False)

In [149]:
grades = pd.read_csv("course_grades.csv")
scores = pd.read_csv("final_regression.csv")

s = sklearn.preprocessing.MinMaxScaler()
grades["avg_gpa"] = s.fit_transform(np.array(grades["avg_gpa"]).reshape(-1,1))
scores["score"] = s.fit_transform(np.array(scores["score"]).reshape(-1,1))

def class_score(professor, course):
    return scores[scores.name == professor]["score"].iloc[0] + grades[grades.course_name == course]["avg_gpa"].iloc[0] / 2

def get_schedule_score(schedule):
    scores = []
    for i in schedule.courses:
        scores.append(class_score(i.professors[0], i.course_name))

    return sum(scores) /  len(scores)